***注意事项*：
- 本文件运行位置：./
- 本文件所属文件夹下内容：
    1. Data：LSH模型所需测试集与训练集
    2. QueryResult：LSH查询测试集产生的查询结果
    3. PingPong：NDSS 2020 PingPong数据集
    4. CsvHub：五元组(Quintuple), 流量特征文件(Features)
    5. device_ip_map.json：IOT设备的IP集

## 基于流包长分布、局部敏感性哈希算法的事件级指纹识别

### （一）原始流量基于五元组的汇编
- 内容概述：五元组（源IP地址、目的IP地址、协议号、源端口、目的端口）用于描述每一个数据包的基本信息。需要将五元组聚合城

In [ ]:
'''五元组流量汇编模块
   输入：PingPong Pcap数据集（含测试集、训练集）
   输出：csv 五元组汇编文件，csv 流量特征文件
'''

import os
import time
from collections import defaultdict
import csv
import io
import struct
from tqdm.notebook import tqdm  #展示进度条
import binascii

error_pcap_name = []  #错误统计
command_Quintuple = "nohup tshark -r {} -T fields -E separator=, -e frame.number -e frame.time_epoch -e frame.len -e eth.src -e eth.dst -e ip.src -e ip.dst -e ip.proto >> {} &"
command_Features = 'tshark -r {} -T fields -E separator=$ -e frame.number -e frame.time_epoch -e eth.type -e ip.src -e ip.dst -e ip.proto -e ip.opt.padding -e ip.opt.ra -e tcp.srcport -e tcp.dstport -e tcp.stream -e tcp.window_size -e tcp.len -e ssl.handshake.ciphersuite -e udp.srcport -e udp.dstport -e udp.stream -e dns.qry.name -e http -e ntp > {}'

#1. 文件预统计
path = r'./PingPong'    #PingPong数据集路径
count = 0
for root,dirs,files in os.walk(path):    #遍历统计
    for each in files:
        if each.endswith(".pcap"):
            count += 1  #统计文件夹下文件个数
print(f"数据集Pcap文件 共计：{count}(个)")  # 输出结果
ShowProcessBar = tqdm(range(count))

#2. Pcap文件处理
for dir,folder,file in os.walk(path):
    for f in file:
        #剔除非pcap文件
        if (f[-5:] != ".pcap"):
            continue  #跳过
        
        file_path = f'{dir}/'+f
        ShowProcessBar.update(1)  #更新进度条
        
        try: 
            #五元组提取
            os.system(command_Quintuple.format(file_path,file_path+'.Quintuple.csv'))
            #流量特征提取
            os.system(command_Features.format(file_path,file_path+'.features.csv'))  #生成features.csv

        except Exception as e:
            print(f"**错误：处理此文件时发生异常{dir}/{f}")
            print('错误信息：'+str(e))
            error_pcap_name.append(f'{dir}/'+f)
            try:
                os.remove(f'{dir}/'+f+'.Quintuple.csv')
            except:
                pass
            continue

print('五元组提取完毕！')

In [ ]:
'''构建CsvHub(1,五元组部分)'''
# 五元组汇编文件（训练/测试集分类）
# 流量特征文件
for root,dirs,file in os.walk('./PingPong/'):
    for each in file:
        if (each.find('detection') != -1 and each.endswith('.csv')):
            os.system(f'cp {root}/{each} ./CsvHub/Quintuple/test-data/')
        elif (each.find('detection') == -1 and each.endswith('.csv')):
            os.system(f'cp {root}/{each} ./CsvHub/Quintuple/train-data/')
        elif (each.endswith('.features.csv')):
            os.system(f'cp {root}/{each} ./CsvHub/Features')


### （二）报文长度概率分布特征向量的提取

#### iot设备IP-MAC地址的提取

In [10]:
'''iot设备IP地址提取模块'''

# 从json中提取设备IP地址
f = open('./device_ip_map.json','r')
eval(f.read())

# 提取每个iot设备的ip（从csv文件中）
DevciesToIp = {}   #iot设备五元组文件(.csv)所对应的ip地址
Path = r'./PingPong'    

'''函数：提取timestamps文件所对应的五元组文件列表'''
def WalkDirCsv(Path):
    FileList = []
    for root,dirs,files in os.walk(Path):
        for each in files:
            if each.endswith("Quintuple.csv"):
                FileList.append(root + '/' + each)
    
    return FileList
                
#提取timestamps对应的csv文件
for root,dirs,files in os.walk(Path):    
    for each in files:
        if each.endswith(".timestamps"):
            if (each in devices_ip):   #剔除无ip记录的五元组文件
                FList = WalkDirCsv(root.split('/timestamps')[0])
                
                for File in FList:
                    DevciesToIp[File] = devices_ip[each]

In [23]:
'''iot设备MAC地址提取模块'''

DevicesMAC = {}

'''函数：常规匹配五元组文件(.csv)中的IP与MAC地址'''
def FindMAC(FilePath):
#读取设备csv
    with open(FilePath,'r') as f:
        while(True):
            EventPacket = f.readline().split(',')
            if (len(EventPacket) == 1):
                f.close()
                raise Exception(f'文件:{FilePath},\n 未找到设备对应的IP信息!')
            
            #寻找IP与MAC关系
            if (EventPacket[5] == DevciesToIp[FilePath]):
                f.close()
                return EventPacket[3]
            elif (EventPacket[6] == DevciesToIp[FilePath]):
                f.close()
                return EventPacket[4] 
        
                            
for root,dirs,files in os.walk(Path):    #遍历统计
    for each in files:
        FilePath = root + '/' + each
        
        if each.endswith(".Quintuple.csv"):
            if (FilePath in DevciesToIp):   #文件有对应设备的IP
                try:
                    DevicesMAC[each] = FindMAC(FilePath)  #找到对应MAC
                except:
                    print(f'Warning：文件{each}，IP未找到。')

print('\nMAC地址表：')
for i in DevicesMAC:
    print(i.split('.')[0],'      ',DevicesMAC[i])


Warning：文件sengled-bulb-intensity.eth1.local.pcap.Quintuple.csv，IP未找到。

MAC地址表：
dlink-plug        90:8d:78:e3:81:0c
dlink-siren        c4:12:f5:de:38:20
hue-bulb-intensity        00:17:88:69:ee:e4
hue-bulb-onoff        00:17:88:69:ee:e4
rachio-sprinkler-quickrun        d0:c5:d3:28:c9:2a
tplink-bulb-color        50:c7:bf:59:d5:84
tplink-bulb-intensity        50:c7:bf:59:d5:84
tplink-bulb-onoff        50:c7:bf:59:d5:84
tplink-plug        50:c7:bf:33:1f:09
wemo-insight-plug        14:91:82:25:10:77
wemo-plug        94:10:3e:36:60:09
amazon-plug        5c:41:5a:ce:36:02
arlo-camera        64:bc:0c:43:3f:40
blossom-sprinkler-mode        64:bc:0c:43:3f:40
blossom-sprinkler-quickrun        28:c2:dd:47:17:b6
dlink-plug        90:8d:78:e3:81:0c
dlink-siren        64:bc:0c:43:3f:40
ecobee-thermostat-fan        64:bc:0c:2a:d2:aa
ecobee-thermostat-hvac        64:bc:0c:2a:d2:aa
kwikset-doorlock        64:bc:0c:43:3f:40
nest-thermostat        64:bc:0c:43:3f:40
rachio-sprinkler-mode        d0:c5:d3:28

#### \*特殊IP(MAC)地址处理
非IOT设备的IP(MAC)地址，如路由器、网关、智能手机等，设计算法与手工结合的方式，选取非此类设备的、在同一设备数据包中出现次数最多的IP(MAC)地址，作为该设备的地址。

#### 概率分布特征向量的提取

In [ ]:
''' 数据集（训练、测试集）的构造与生成模块 '''

import math
import pickle
import os
from tqdm import tqdm

DIRECTION_IN = 0x00
DIRECTION_OUT = 0x01


class UNSWDataset(object):
    # 待识别的iot设备MAC地址集(提取好的)
    DEVICE_IOT_LIST = '''
        kwikset-doorlock          90:8d:78:e3:81:0c
        nest-thermostat          64:16:66:1f:f0:3e
        roomba-vacuum-robot          d0:c5:d3:90:54:73
        sengled-bulb-onoff          b0:ce:18:27:4d:ab
        tplink-bulb-color          50:c7:bf:59:d5:84
        dlink-plug          90:8d:78:e3:81:0c
        dlink-siren          c4:12:f5:de:38:20
        hue-bulb          00:17:88:69:ee:e4
        rachio-sprinkler          d0:c5:d3:28:c9:2a
        tplink-plug          50:c7:bf:33:1f:09
        wemo-insight-plug          14:91:82:23:cc:41
        wemo-plug          94:10:3e:36:60:09
        amazon-plug          5c:41:5a:ce:36:02
        blossom-sprinkler          28:c2:dd:47:17:b6
        ecobee-thermostat          64:bc:0c:2a:d2:aa
        ring-alarm          f4:84:4c:45:69:07
        blink-camera-photo          f4:b8:5e:fe:19:bc
    '''.split()
    
    # 非iot设备的MAC地址集（智能终端等）
    DEVICE_NON_IOT_LIST = '''
    '''.split()

    def __init__(self):
        self.device_list, self.iot_list, self.non_iot_list = {}, {}, {}
        for i in range(0, len(UNSWDataset.DEVICE_IOT_LIST), 2):
            self.iot_list[UNSWDataset.DEVICE_IOT_LIST[i]] = UNSWDataset.DEVICE_IOT_LIST[i + 1]
            self.device_list[UNSWDataset.DEVICE_IOT_LIST[i]] = UNSWDataset.DEVICE_IOT_LIST[i + 1]
        for i in range(0, len(UNSWDataset.DEVICE_NON_IOT_LIST), 2):
            self.non_iot_list[UNSWDataset.DEVICE_NON_IOT_LIST[i]] = \
                UNSWDataset.DEVICE_NON_IOT_LIST[i + 1]
            self.device_list[UNSWDataset.DEVICE_NON_IOT_LIST[i]] = \
                UNSWDataset.DEVICE_NON_IOT_LIST[i + 1]
        self.mac_device_map = {v: k for k, v in self.device_list.items()}
        self.label_map = {mac: i + 1 for i, mac in enumerate(self.mac_device_map.keys())}
        
        #获取所有csv文件列表
        self.FEATURE_PATH = './CsvHub/Features'

    '''函数：读取五元组与流数据'''
    def data_generator(self,Path):
        self.FILE_PATH = os.listdir(Path)
        
        print('开始读取五元组数据...')
        
        for FileName in tqdm(self.FILE_PATH,desc='五元组文件'):
            #跳过非csv文件
            if not FileName.endswith('.Quintuple.csv'):
                continue
            try:
                #获取真实路径
                file_path = Path + '/' + FileName
                feature_path = self.FEATURE_PATH + '/' + FileName.split('.Quintuple.csv')[0] + '.features.csv'
                f = open(file_path, 'r')
                g = open(feature_path, 'r')
                l1, l2 = f.readline(), g.readline()
                print(file_path)
                while l1 and l2:
                    fields1 = l1.split(',')
                    fields2 = l2.split('$')
                    timestamp, size, eth_src, eth_dst = float(fields1[1]), int(fields1[2]), fields1[3], fields1[4]
                    tcp_stream, udp_stream = fields2[-10], fields2[-4]
                    yield timestamp, size, eth_src, eth_dst, tcp_stream, udp_stream
                    l1, l2 = f.readline(), g.readline()
                f.close()
                g.close()
                if l1 or l2:
                    print('Error: line count does not match')
                print('finish reading ..')
            except:
                print(f'错误:{file_path}')
                print(f'Details:出错行_fields1:{fields1}_fields2{fields2}\n')
    
    '''函数：数据集生成(1)'''
    def get_dataset(self,Path):
        
        print('开始构造数据集...')
        
        dataset = {mac: [] for mac in self.device_list.values()}
        flow_set = {mac: {} for mac in self.device_list.values()}

        flow_instance_duration = 10  # seconds
        flow_duration = 2  # seconds

        def on_new_data_point(sample, mac):
            total_count = len(sample['packet_series'])
            packet_counter = {}
            for p in sample['packet_series']:
                packet_counter[p] = packet_counter.get(p, 0) + 1
            distribution = {pl: count / total_count for pl, count in packet_counter.items()}
            # print(distribution)
            dataset[mac].append({'t': sample['packet_time'][0], 'fv': distribution})

        # flow_record {'start_time': int, 'last_time': int, packet_series: [], packet_time: []}
        def on_new_packet(packet_tuple, timestamp, mac, stream_id):
            if stream_id not in flow_set[mac]:
                flow_set[mac][stream_id] = {
                    'packet_series': [packet_tuple],
                    'packet_time': [timestamp]
                }
            else:
                if timestamp - flow_set[mac][stream_id]['packet_time'][0] > flow_instance_duration or \
                        timestamp - flow_set[mac][stream_id]['packet_time'][-1] > flow_duration:
                    on_new_data_point(flow_set[mac][stream_id], mac)
                    # sliding window
                    while flow_set[mac][stream_id]['packet_time'] and \
                            (timestamp - flow_set[mac][stream_id]['packet_time'][0] > flow_instance_duration or
                             timestamp - flow_set[mac][stream_id]['packet_time'][-1] > flow_duration):
                        flow_set[mac][stream_id]['packet_series'] = flow_set[mac][stream_id]['packet_series'][1:]
                        flow_set[mac][stream_id]['packet_time'] = flow_set[mac][stream_id]['packet_time'][1:]
                flow_set[mac][stream_id]['packet_series'].append(packet_tuple)
                flow_set[mac][stream_id]['packet_time'].append(timestamp)

        for t, size, eth_src, eth_dst, tcp_stream, udp_stream in self.data_generator(Path):
            if not tcp_stream and not udp_stream:
                continue
            stream_id = 't' + tcp_stream if tcp_stream else 'u' + udp_stream
            if eth_src in self.device_list.values():
                packet_label = (size, DIRECTION_OUT)
                on_new_packet(packet_label, t, eth_src, stream_id)
            if eth_dst in self.device_list.values():
                packet_label = (size, DIRECTION_IN)
                on_new_packet(packet_label, t, eth_dst, stream_id)
        return dataset
    
    '''函数：存储数据集'''
    def store_dataset(self):
        
        print('开始导出数据集...')
        
        #生成测试、训练集
        train_set = self.get_dataset('./CsvHub/Quintuple/train-data')
        test_set = self.get_dataset('./CsvHub/Quintuple/test-data')

        simple_record = {k: set() for k in train_set.keys()}
        valid_train_set = {k: [] for k in train_set.keys()}
        for mac, instances in train_set.items():
            for i in instances:
                if len(i['fv']) <= 3:
                    simple_record[mac].add(tuple(i['fv'].keys()))
                else:
                    valid_train_set[mac].append(i)

        simple_record_path = './Data/simple.pkl'
        train_flow_path = './Data/train-flow.pkl'
        test_data_path = './Data/test-data.pkl'
        pickle.dump(simple_record, open(simple_record_path, 'wb'))
        pickle.dump(valid_train_set, open(train_flow_path, 'wb'))
        pickle.dump(test_set, open(test_data_path, 'wb'))


def normalize(sample):
    sqrt2 = math.sqrt(2)
    vector = []
    for direction in [0x00, 0x01]:
        for length in range(20, 1521):
            vector.append(sample.get((length, direction), 0.0))
    return list(map(lambda x: math.sqrt(x) / sqrt2, vector))


if __name__ == '__main__':
    unsw_dataset = UNSWDataset()
    unsw_dataset.store_dataset()


### （三）基于LSH的查询器的实现

#### 局部敏感性哈希（LSH）查询器

In [ ]:
import falconn
import numpy as np

class LSHClassifier(object):
    def __init__(self):
        self.mac_index_map = {}
        self.sample_instance_set = []
        self.lsh_index = None

    def load_training_data(self, training_data):
        start_index = 0
        for device_mac, instances in training_data.items():
            self.mac_index_map[device_mac] = (start_index, start_index + len(instances))
            start_index += len(instances)
            self.sample_instance_set.extend(instances)

        parameter = falconn.get_default_parameters(len(self.sample_instance_set), len(self.sample_instance_set[0]))
        self.lsh_index = falconn.LSHIndex(parameter)
        self.lsh_index.setup(np.array(self.sample_instance_set, dtype=np.float32))

    def get_classification_result(self, sample):
        q = self.lsh_index.construct_query_object()
        index = q.find_nearest_neighbor(np.array(sample, dtype=np.float32))
        for k, v in self.mac_index_map.items():
            left_bound, right_bound = v
            if left_bound <= index < right_bound:
                return k, index
        print("fail to get classification result, index = ", index)
        return None, None


#### 训练集预处理

In [ ]:
import pickle
train_set = pickle.load(open('./Data/train-flow.pkl', 'rb'))
print(train_set.keys())

import math

def calculate_hellinger_distance(v1, v2):
    s1 = set(v1.keys())
    s2 = set(v2.keys())
    s = s1 | s2
    d = 0.0
    for packer_header in s:
        p1 = v1.get(packer_header, 0.0)
        p2 = v2.get(packer_header, 0.0)
        d += (math.sqrt(p1) - math.sqrt(p2)) ** 2
    d = math.sqrt(d) / math.sqrt(2)
    return d


def merge_sample(v1, v2, w):
    S = {}
    s1 = set(v1.keys())
    s2 = set(v2.keys())
    s = s1 | s2
    for packet_header in s:
        p1 = v1.get(packet_header, 0.0)
        p2 = v2.get(packet_header, 0.0)
        p = (p1 * w + p2) / (w + 1)
        S[packet_header] = p
    return S
        

# data reduction

new_dataset = {}
for k, v in train_set.items():
    sample_space_map = {}
    total_count = len(v)
    for instance in v:
        if tuple(instance['fv'].keys()) in sample_space_map:
            original_bucket = sample_space_map[tuple(instance['fv'].keys())]
            bucket, found = [], False
            for index, (sample, n) in enumerate(original_bucket):
                d = calculate_hellinger_distance(instance['fv'], sample)
                if d < 1e-3:
                    new_sample = merge_sample(sample, instance['fv'], n)
                    bucket.append((new_sample, n + 1))
                    found = True
                    bucket.extend(original_bucket[index + 1:])
                    break
                else:
                    bucket.append((sample, n))
            if not found:
                bucket.append((instance['fv'], 1))
            sample_space_map[tuple(instance['fv'].keys())] = bucket
        else:
            sample_space_map[tuple(instance['fv'].keys())] = [(instance['fv'], 1)]
    print(k, total_count, len(sample_space_map))
    new_dataset[k] = sample_space_map
    
pickle.dump(new_dataset, open('./Data/new-train-flow.pkl', 'wb'))

#### 模型训练与预测

In [ ]:
import pickle

import data
import lsh_classifier
from tqdm import tqdm

def offline_verification_unsw_new():
    unsw_dataset = data.UNSWDataset()
    classifier = lsh_classifier.LSHClassifier()
    
    # get training dataset
    print('开始进行模型训练...')
    flow_data_path = './Data/new-train-flow.pkl'
    simple_record_path = './Data/simple.pkl'
    training_set = pickle.load(open(flow_data_path, 'rb'))
    simple_record = pickle.load(open(simple_record_path, 'rb'))
    normalized_dataset = {k: [] for k in training_set.keys()}
    for mac, instances in tqdm(training_set.items()):
        if mac in unsw_dataset.iot_list.values():
            for samples in instances.values():
                for s in samples:
                    fv, _ = s
                    normalized_dataset[mac].append(data.normalize(fv))
    classifier.load_training_data(normalized_dataset)
    
    #测试集
    print('开始运行测试集...')
    test_data_path = './Data/test-data.pkl'
    test_set = pickle.load(open(test_data_path, 'rb'))
    for mac, test_instances in tqdm(test_set.items()):
        result_record_path = './QueryResult/' + mac.replace(':', '-') + '.txt'
        f = open(result_record_path, 'w')
        device_simple_record = simple_record[mac]
        for test_instance in test_instances:
            if len(test_instance['fv']) <= 3:
                if tuple(test_instance['fv'].keys()) in device_simple_record:
                    f.write("{} {}\n".format(test_instance['t'], 0))
                else:
                    f.write("{} {}\n".format(test_instance['t'], -1))
            else:
                result, index = classifier.get_classification_result(data.normalize(test_instance['fv']))
                if result != None:
                    label = unsw_dataset.label_map[result]
                    f.write("{} {} {}\n".format(test_instance['t'], label, index))
                else:
                    f.write("{} {}\n".format(test_instance['t'], -2))


if __name__ == '__main__':
    offline_verification_unsw_new()


### （四）数据分析

####  测试集命中率计算

In [10]:
Accuracys = {}
import os
for root,dirs,file in os.walk('./QueryResult'):
    for each in file:
        if each.endswith('.txt') and each.find('checkpoint') == -1:
            FilePath = root + '/' + each
            No_Hit = 0
            Count = 0
            with open(FilePath,'r') as f:
                ResultList = f.read().strip().split('\n')
                for Result in ResultList:
                    Result = Result.split()
                    #print(Result)
                    Count += 1
                    if (int(Result[-1]) == -1 or int(Result[-1]) == -2):
                        No_Hit += 1
                
                f.close()
            
                Accuracys[each.split('.txt')[0]] = (Count-No_Hit)/Count
                print(each.split('.txt')[0],'   *命中率：',round(Accuracys[each.split('.txt')[0]]*100,3),'%')
                
# 平均命中率
AvgAcc = 0
for Devices in Accuracys:
    AvgAcc = AvgAcc + Accuracys[Devices]/len(Accuracys)
print('\n平均命中率：',round(AvgAcc*100,3),'%')

d0-c5-d3-28-c9-2a    *命中率： 99.446 %
50-c7-bf-33-1f-09    *命中率： 97.259 %
94-10-3e-36-60-09    *命中率： 99.61 %
5c-41-5a-ce-36-02    *命中率： 100.0 %
28-c2-dd-47-17-b6    *命中率： 99.997 %
64-bc-0c-2a-d2-aa    *命中率： 91.015 %
f4-84-4c-45-69-07    *命中率： 99.64 %
90-8d-78-e3-81-0c    *命中率： 99.995 %
64-16-66-1f-f0-3e    *命中率： 99.46 %
d0-c5-d3-90-54-73    *命中率： 99.949 %
50-c7-bf-59-d5-84    *命中率： 99.829 %
c4-12-f5-de-38-20    *命中率： 99.992 %
00-17-88-69-ee-e4    *命中率： 99.93 %

平均命中率： 98.932 %
